In [ ]:
########################################
## import all the necessary libraries ##
########################################

import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from collections import defaultdict

In [ ]:
########################################################
##        read the file that we want to search        ##
########################################################
g_group = defaultdict(lambda: [])
with open('/Users/vasou/Documents/HLA/g_group.txt') as f:
    for line in f.read().split('\n'):
        if '#' in line or line == '':
            continue
        result = line.split("*;")[1]
        result = ';'.join(result.split(';')[:-1]) if result.split(';')[-1] == '' else result
        g_group[line.split("*;")[0]].append(result)
        
        
decoder = pd.read_csv(f'/Users/vasou/Documents/HLA/decode.txt', sep='\t')

In [ ]:
def decoding(data, decoder):
    
    for column, gene in zip(data.columns,['A', 'A', 'B', 'B', 'C', 'C', 'DRB1', 'DRB1', 'DQB1','DQB1','DPB1','DPB1']):
        decoded = []
        print(column)

        alleles = data[column].values.tolist()
        
        for allele in tqdm(alleles):
            
            allele = allele.strip()
            
            if allele.endswith(':XX'):
                decoded.append('no entry')
                continue
                
            if len(allele) == 2:
                decoded.append(allele)
                continue
                
            if allele.endswith('01G') or allele.endswith('02G'):
                decoded.append('already done')
                continue
                
            if allele == 'not found':
                decoded.append('no entry')
                continue
                
            if re.search('[a-zA-Z]', allele):

                ######################################################
                ##                decode allele                     ##
                ######################################################
                
                try:
                    found = decoder[decoder['CODE'] == allele.split(':')[1]]
                    found['SUBTYPE'].tolist()[0]
                except:
                    #print('code not found')
                    decoded.append(allele)
                    continue
                
                if found['*'].tolist()[0] == '*':
                    decoded_allele = found['SUBTYPE'].tolist()[0]
                    decoded.append(decoded_allele)
                    
                else:
                    subtype = found['SUBTYPE'].tolist()[0]
                    subtype_split = subtype.split('/')
                    prefix = allele.split(':')[0]
                    decoded_allele = '/'.join([prefix + ':' + i for i in subtype_split])
                    decoded.append(decoded_allele)
            else:

                decoded.append('i dont need decoding')
                
        data[column + '_decoded'] = decoded
        
    return data


#results = decoding(cyprus_CBUs, decoder)
#results = decoding(cyprus, decoder)


In [ ]:
def grouping_funcion(g_group, allele, strict=True, level=0):

    for element in g_group:
        if ';' in element:
            splitted_element = element.split(';')
            for item in splitted_element[0].split('/'):
                if strict and item.startswith(allele) and len(item.replace(allele, "").replace(":01","")) == 0 :
                    return splitted_element[1]
                elif not strict and item.startswith(allele):
                    return splitted_element[1]
        else:
            
            if strict and element.startswith(allele) and len(element.replace(allele, "").replace(":01","")) == 0 :
                return allele
            elif not strict and element.startswith(allele):
                return allele
    if level==0:
        return grouping_funcion(g_group, allele[:-3], strict=strict, level=1)
    return None


def grouping(decoded_data, g_group):
    
    need_col = [column for column in decoded_data.columns if 'decoded' in column]
    
    for dec, gene in zip(decoded_data[need_col].columns, ['A', 'A', 'B', 'B', 'C', 'C', 'DRB1', 'DRB1', 'DQB1','DQB1','DPB1','DPB1']):
        
        group_dec = []
        for i in tqdm(decoded_data.iterrows()):
            
            if 'already' in i[1][dec]:
                group_dec.append(i[1][dec.split('_decoded')[0]])
                continue
                
            if 'no entry' in i[1][dec]:
                group_dec.append('no entry')
                continue
                
            if len(i[1][dec]) == 2:
                group_dec.append(i[1][dec])
                continue
            
            if 'decoding' in i[1][dec]:
                if 'new' in i[1][dec]:
                    al = dec.split('_new')[0]
                    allele = i[1][al+'_new']
                else:
                    allele = i[1][dec.split('_decoded')[0]]

                flag = allele[-1].isdigit()
                if flag:
                    allele = allele + ':01'
                
                #if len(allele) != 8 and len(allele) != 11:
                 #   allele = allele + ':01'
                    
                element = grouping_funcion(g_group[gene], allele, level=0 if flag else 1 )
                if element is None:
                    element = grouping_funcion(g_group[gene], allele, level=0 if flag else 1, strict=False)

                if element is None:
                    group_dec.append(f'not found in g-group: {allele}')
                else:
                    group_dec.append(element)
                    

            else:
                dd = []
                for splitted_allele in i[1][dec].split('/'):
                    flag = splitted_allele[-1].isdigit()
                    if flag:
                        splitted_allele = splitted_allele + ':01'

                    ##########################################
                    ##              G-grouping              ##
                    ##########################################

                    element = grouping_funcion(g_group[gene], splitted_allele, level=0 if flag else 1 )
                    if element is None:
                        element = grouping_funcion(g_group[gene], splitted_allele, level=0 if flag else 1, strict=False)

                    if element is None:
                        print(splitted_allele)
                        dd.append(f'not found in g-group: {splitted_allele}')
                    else:
                        dd.append(element)
                if len(set(dd)) == 1:
                    dd = dd[0]
                else:
                    dd = '/'.join(dd)
                group_dec.append(dd)  
        #print(dec)

        decoded_data[dec.split('_decoded')[0] + '_grouped'] = group_dec
    
    return decoded_data
    
#tmp = grouping(cyprus_BMD_decoded, g_group)
#grouped_results = grouping(cyprus_CBUs_decoded, g_group)
#tmp = grouping(crete_decoded, g_group)


In [ ]:
results = decoding(patra, decoder)
results.to_excel('data_decoded.xlsx')
patra_decoded = pd.read_excel(f'/Users/vasou/Documents/HLA/patra_decoded.xlsx')
patra_decoded = patra_decoded.replace(r'\?.*','not found', regex = True)
patra_decoded = patra_decoded.replace(np.nan,'not found')

tmp = grouping(patra_decoded, g_group)
tmp.to_excel('patra_grouped.xlsx')